In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import math

In [2]:
df_Market_Cap = pd.read_pickle("UsableData/MarketCap.pkl")
df_Employees = pd.read_pickle("UsableData/Employees.pkl")
df_Book_Value = pd.read_pickle("UsableData/BookValue.pkl")
df_Returns = pd.read_pickle("UsableData/Returns.pkl").T

In [3]:
df_eligible = df_Market_Cap.copy(deep=True)
for column in df_eligible:
    df_eligible[column].astype("bool")
    df_eligible[column] = False

In [4]:
def unusableReturns(ric, year):
    selection = df_Returns[ric][(df_Returns.index >= f"{year}-01-01") & (df_Returns.index <= f"{year}-12-31")]
    return bool(selection.isna().values.any())

In [5]:
years = df_Market_Cap.columns.to_numpy()[:-1]
for ric in df_Market_Cap.index:
    for year in years:
        mc = df_Market_Cap.loc[ric, year]
        emp = df_Employees.loc[ric, year]
        bv = df_Book_Value.loc[ric, year]
        if(math.isnan(mc) or math.isnan(emp) or math.isnan(bv) or unusableReturns(ric, year+1)):
            df_eligible.at[ric, year+1] = False
        else:
            df_eligible.at[ric, year+1] = True
df_eligible.drop(columns=[years[0]], inplace=True) # Drop first and last year
df_eligible.columns

Index([1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021],
      dtype='int32', name='Year')

In [6]:
for i in range(len(df_eligible.columns)):
    df = df_eligible[df_eligible[df_eligible.columns[i]] == True]
    print(f"{df_eligible.columns[i]}: {len(df.index)}")

1996: 2
1997: 22
1998: 33
1999: 103
2000: 192
2001: 292
2002: 346
2003: 360
2004: 431
2005: 442
2006: 492
2007: 553
2008: 585
2009: 582
2010: 552
2011: 577
2012: 588
2013: 551
2014: 539
2015: 470
2016: 451
2017: 439
2018: 449
2019: 461
2020: 490
2021: 523


In [7]:
df_eligible.to_pickle("UsableData/EligibleStocks.pkl")